# World Cup Socioeconomic Choropleth

Interactively explore GDP per capita, HDI, life expectancy, and mean years of schooling for World Cup participant countries.


In [1]:
from pathlib import Path

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display


In [2]:
PROJECT_ROOT = Path(__file__).resolve().parents[2] if "__file__" in globals() else Path.cwd()
while not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
DATA_DIR = PROJECT_ROOT / "data" / "created_datasets" / "socioeconomic"

metric_sources = {
    "GDP per capita": ("gdp_world_cup.csv", "gdp_per_capita"),
    "HDI": ("hdi_world_cup.csv", "hdi"),
    "Life expectancy": ("life_expectancy_world_cup.csv", "life_expectancy"),
    "Mean years of schooling": ("schooling_world_cup.csv", "mean_school_years"),
}

frames = []
for label, (filename, value_col) in metric_sources.items():
    path = DATA_DIR / filename
    df = pd.read_csv(path)
    subset = df[["country", "iso3", "year", value_col]].copy()
    subset.rename(columns={value_col: "value"}, inplace=True)
    subset["metric"] = label
    frames.append(subset)

metrics_df = pd.concat(frames, ignore_index=True)
metrics_df["year"] = metrics_df["year"].astype(int)
year_min, year_max = int(metrics_df["year"].min()), int(metrics_df["year"].max())
metrics_df.head()


,country,iso3,year,value,metric
0,Angola,AGO,1990,965.866840,GDP per capita
1,Argentina,ARG,1990,4315.334031,GDP per capita
2,Australia,AUS,1990,18249.301454,GDP per capita
3,Austria,AUT,1990,21596.068220,GDP per capita
4,Belgium,BEL,1990,20600.375279,GDP per capita


In [3]:
def build_choropleth(metric_label: str, year: int):
    slice_df = metrics_df[
        (metrics_df["metric"] == metric_label) & (metrics_df["year"] == year)
    ]
    if slice_df.empty:
        fig = px.choropleth()
        fig.add_annotation(text="No data for this selection", showarrow=False)
        return fig

    range_color = (slice_df["value"].min(), slice_df["value"].max())
    fig = px.choropleth(
        slice_df,
        locations="iso3",
        color="value",
        hover_name="country",
        color_continuous_scale="Viridis",
        labels={"value": metric_label},
    )
    fig.update_layout(
        title=f"{metric_label} ({year})",
        coloraxis_colorbar_title=metric_label,
        margin=dict(l=0, r=0, t=50, b=0),
    )
    fig.update_coloraxes(cmin=range_color[0], cmax=range_color[1])
    return fig


In [ ]:
metric_dropdown = widgets.Dropdown(
    options=sorted(metric_sources.keys()),
    value="GDP per capita",
    description="Metric:",
    layout=widgets.Layout(width="300px"),
)

year_slider = widgets.IntSlider(
    min=year_min,
    max=year_max,
    step=1,
    value=year_min,
    description="Year",
    continuous_update=False,
    readout=True,
)

fig_output = widgets.Output()


def update_plot(change=None):
    with fig_output:
        fig_output.clear_output(wait=True)
        fig = build_choropleth(metric_dropdown.value, year_slider.value)
        fig.show()


metric_dropdown.observe(update_plot, names="value")
year_slider.observe(update_plot, names="value")

controls = widgets.VBox([metric_dropdown, year_slider])
ui = widgets.HBox([controls])

update_plot()
display(widgets.VBox([ui, fig_output]))
